# 数据合并处理

In [11]:
import pandas as pd
import numpy as np
import scipy as sp

def read_csv_file(f, logging = False):
    data = pd.read_csv(f)
    if logging:
        print(data.head())
        print(data.columns.values)
        print(data.describe())
        print(data.info())
    return data

def categories_process_first_class(cate):
    cate = str(cate)
    return int(cate[0])
        
    
def categories_process_second_class(cate):
    cate = str(cate)
    if len(cate) < 3:
        return 0
    else:
        return int(cate[1:])
    
def age_process(age):
    age = int(age)
    if age == 0:
        return 0
    if age < 16:
        return 1
    if age < 23:
        return 2
    if age < 27:
        return 3
    else:
        return 4

def province_process(hometown):
    hometown = str(hometown)
    if len(hometown) == 1:
        return 0
    else:
        return int(hometown[: -2])

def city_process(hometown):
    hometown = str(hometown)
    if len(hometown) == 1:
        return 0
    else:
        return int(hometown[-2: ])

def get_time_day(t):
    t = str(t)
    t = int(t[0: 2])
    return t

def get_time_hour(t):
    t = str(t)
    t = int(t[2: 4])
    if t < 6:
        return 0
    if t < 12:
        return 1
    if t < 18:
        return 2
    else:
        return 3

def get_time_min(t):
    t = str(t)
    t = int(t[4: ])
    return t

def logloss(act, pred):
    epsilon = 1e-5
    pred = sp.maximum(epsilon, pred)
    pred = sp.minimum(1 - eplison, pred)
    ll = sum(act * sp.log(pred) + sp.subtract(1, act) * sp.log(sp.subtract(1, pred)))
    ll = ll * -1.0/len(act)
    return ll

In [12]:
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

print('test dataset')
test_data = read_csv_file('./pre/test.csv', logging = True)
print('================================================\n')
print('train dataset:')
train_data = read_csv_file('./pre/train.csv', logging = True)
print('================================================\n')
print('ad dataset')
ad = read_csv_file('./pre/ad.csv', logging = True)
print('================================================\n')
print('app dataset')
app_categories = read_csv_file('./pre/app_categories.csv', logging = True)
print('================================================\n')
print('user dataset')
user = read_csv_file('./pre/user.csv', logging = True)
print('================================================\n')
print('position dataset')
position = read_csv_file('./pre/position.csv', logging= True)
print('================================================\n')
print('user_app_actions dataset')
user_app_actions = read_csv_file('./pre/user_app_actions.csv', logging = True)

test dataset
   instanceID  label  clickTime  creativeID   userID  positionID  \
0           1     -1     310000        3745  1164848        3451   
1           2     -1     310000        2284  2127247        1613   
2           3     -1     310000        1456  2769125        5510   
3           4     -1     310000        4565     9762        4113   
4           5     -1     310000          49  2513636        3615   

   connectionType  telecomsOperator  
0               1                 3  
1               1                 3  
2               2                 1  
3               2                 3  
4               1                 3  
['instanceID' 'label' 'clickTime' 'creativeID' 'userID' 'positionID'
 'connectionType' 'telecomsOperator']
          instanceID     label      clickTime     creativeID        userID  \
count  338489.000000  338489.0  338489.000000  338489.000000  3.384890e+05   
mean   169245.000000      -1.0  311479.490613    3001.534765  1.409519e+06   
std     9

   userID  installTime  appID
0       1       182032    933
1       5       141354    589
2       5       251727    565
3       5       260932    596
4       5       261318    536
['userID' 'installTime' 'appID']
             userID   installTime         appID
count  6.003471e+06  6.003471e+06  6.003471e+06
mean   1.403576e+06  1.483695e+05  5.125716e+03
std    8.074011e+05  9.097316e+04  2.118739e+04
min    1.000000e+00  1.000000e+04  3.540000e+02
25%    7.055445e+05  6.164500e+04  5.890000e+02
50%    1.404560e+06  1.408000e+05  8.270000e+02
75%    2.100952e+06  2.320160e+05  1.969000e+03
max    2.805117e+06  3.023590e+05  4.332670e+05
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6003471 entries, 0 to 6003470
Data columns (total 3 columns):
userID         int64
installTime    int64
appID          int64
dtypes: int64(3)
memory usage: 137.4 MB
None


In [ ]:
print('================================================\n')
print('user_installedapps dataset')
user_installedapps = read_csv_file('./pre/user_installedapps.csv', logging = False)

In [ ]:
train = train_data.drop(['label', 'conversionTime'], axis = 1)
test = test_data.drop(['label', 'instanceID'], axis = 1)
data = pd.concat((train, test), axis = 0)

In [ ]:
print(data.shape)
print(train.shape)
print(test.shape)

In [ ]:
data.isnull().sum()

In [ ]:
data_ad = data.merge(ad, on = 'creativeID', how = 'left')
data_ad.shape

In [ ]:
data_ad['click_day'] = data_ad['clickTime'].apply(get_time_day)
data_ad['click_min'] = data_ad['clickTime'].apply(get_time_min)
data_ad['click_hour'] = data_ad['clickTime'].apply(get_time_hour)
data_ad.drop(['clickTime'], inplace = True, axis = 1)
data_ad.tail()

In [ ]:
data_ad_app = data_ad.merge(app_categories, on = 'appID', how = 'left')
data_ad_app['app_first_categories'] = data_ad_app['appCategory'].apply(categories_process_first_class)
data_ad_app['app_second_categories'] = data_ad_app['appCategory'].apply(categories_process_second_class)
data_ad_app.drop(['appCategory'], axis = 1, inplace = True)

In [ ]:
data_ad_app_user = data_ad_app.merge(user, on = 'userID', how = 'left')

In [ ]:
##test数据集的age数据缺失较多
data_ad_app_user['age'].describe()

In [ ]:
data_ad_app_user['age'].replace(0.0, data_ad_app_user['age'][data_ad_app_user['age'] != 0.0].mean(), inplace = True)

In [ ]:
data_ad_app_user['age'].fillna(data_ad_app_user['age'][data_ad_app_user['age'] != 0.0].mean(), inplace = True)
data_ad_app_user['age'].isnull().sum()

In [ ]:
data_ad_app_user['age'] = data_ad_app_user['age'].apply(age_process)

In [ ]:
data_ad_app_user['residence_province'] = data_ad_app_user['residence'].apply(province_process)
data_ad_app_user['residence_city'] = data_ad_app_user['residence'].apply(city_process)
data_ad_app_user.drop(['residence'], inplace = True, axis = 1)
data_ad_app_user.tail()

In [ ]:
data_ad_app_user_position = data_ad_app_user.merge(position, on = 'positionID', how = 'left')

In [ ]:
x_train = data_ad_app_user_position[: 3749528]
y_train = train_data['label']
test = data_ad_app_user_position[3749528: ]

In [ ]:
import pickle
with open('values.pkl', 'wb') as f:
    pickle.dump(x_train, f)
    pickle.dump(y_train, f)
    pickle.dump(test, f)

# 数据建模

## 导入数据

In [1]:
import pickle
with open('values.pkl', 'rb') as f:
    x_train = pickle.load(f)
    y_train = pickle.load(f)
    test = pickle.load(f) 

In [ ]:
y_train.value_counts()
##正负样本非常不均衡，但由于过采样后数据量过大，电脑内存有限，使用欠采样的方法

In [4]:
x_train.head()

,creativeID,userID,positionID,connectionType,telecomsOperator,adID,camgaignID,advertiserID,appID,appPlatform,...,gender,education,marriageStatus,haveBaby,hometown_province,hometown_city,residence_province,residence_city,sitesetID,positionType
0,3089,2798058,293,1,1,1321,83,10,434,1,...,2,1,3,1,0,0,13,1,0,3
1,1259,463234,6161,1,2,1535,685,80,14,2,...,2,1,1,0,0,0,2,13,0,1
2,4465,1857485,7434,4,1,147,460,3,465,1,...,1,1,0,0,0,0,15,2,0,1
3,1004,2038823,977,1,1,411,564,3,465,1,...,0,0,0,0,0,0,10,1,0,1
4,1887,2015141,3688,1,1,369,144,84,360,1,...,1,2,2,0,10,1,10,1,0,1


In [12]:
x_train[['click_day', 'click_min', 'click_hour', 'app_first_categories', 'app_second_categories', 'age']].head()

,click_day,click_min,click_hour,app_first_categories,app_second_categories,age
0,17,0,0,1,8,3
1,17,0,0,2,0,2
2,17,0,0,2,9,2
3,17,0,0,2,9,0
4,17,0,0,2,1,4


## 过采样后存储数据

In [ ]:
from imblearn.over_sampling import ADASYN

x_train_oversam, y_train_oversam = ADASYN().fit_sample(x_train, y_train)

In [ ]:
import pickle
with open('values_resam.pkl', 'wb') as f:
    pickle.dump(x_train_oversam, f)
    pickle.dump(y_train_oversam, f)   

In [ ]:
print(x_train_oversam.shape)
print(y_train_oversam.shape)

In [ ]:
## EasyEnsemble
'''
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold

bbc = BalancedBaggingClassifier(
    base_estimator = DecisionTreeClassifier(),
    ratio = 'auto',
    replacement = False,
    random_state = 100,
    n_jobs = -1
)

kf = KFold(n_splits = 5)
for train_index, val_index in kf.split(x_train, y_train):
    X_train,Y_train = x_train[train_index, : ], y_train[train_index]
    X_val, Y_val = x_train[val_index, : ], y_train[val_index]
    bbc.fit(X_train, Y_train)
    pred_bbc = bbc.predict_proba(X_val)
    print(pred_bbc)
    print('The logloss is ', logloss(Y_val, pred_bbc))
'''

## 欠采样并存储数据

In [ ]:
import numpy as np
import pickle
from imblearn.under_sampling import RandomUnderSampler

with open('values.pkl', 'rb') as f:
    x_train = pickle.load(f)
    y_train = pickle.load(f)
    test = pickle.load(f)
    
#x_train = np.array(x_train)
#y_train = np.array(y_train)
#test = np.array(test)

In [18]:
y_train = train_data['label']
y_train = pd.DataFrame(y_train, columns = ['label'])

In [21]:
from imblearn.under_sampling import RandomUnderSampler

bc = RandomUnderSampler(random_state = 100)
x_train_resam, y_train_resam = bc.fit_sample(x_train, y_train)

with open('values_undersampling.pkl', 'wb') as f:
    pickle.dump(x_train_resam, f)
    pickle.dump(y_train_resam, f)
    pickle.dump(test, f)

In [ ]:
x_train_resam = pd.DataFrame(x_train_resam, columns = x_train.columns)
y_train_resam = pd.DataFrame(y_train_resam, columns = ['label'])
print(x_train_resam)

import pickle

with open('lgb_values.pkl', 'wb') as f:
    pickle.dump(x_train_resam, f)
    pickle.dump(y_train_resam, f)
    pickle.dump(test, f)

## 载入数据

In [22]:
import pickle

with open('values_undersampling.pkl', 'rb') as f:
    x_train_resam = pickle.load(f)
    y_train_resam = pickle.load(f)
    test = pickle.load(f)

In [23]:
print(x_train_resam)

[[   4565  156503    3789 ...       8       0       1]
 [   4565 2159098    2891 ...      21       0       1]
 [   2808 1140159    2868 ...       4       0       1]
 ...
 [   1675 1445513    2188 ...       1       0       1]
 [   4077 1252796    5539 ...       0       1       0]
 [   2038 2216718    1865 ...       6       0       2]]


### Xgbosst

#### 用sklearn API

In [ ]:
'''
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
import numpy as np

param_grid = {
    'max_depth': [3, 4, 5, 8, 10],
    'n_estimators': [50, 100, 200, 400, 600, 800, 1000],
    'laerning_rate': [0.1, 0.2, 0.3],
    'gamma': [0, 0.2],
    'subsample': [0.8, 1],
             }

xgb_model = xgb.XGBClassifier()
rgs = GridSearchCV(xgb_model, param_grid, n_jobs = -1, cv = 5)
rgs.fit(x_train_resam, y_train_resam)
print(rgs.best_score_)
print(rgs.best_params_)
pred = rgs.predict_proba(test)
print('The logloss is ', logloss(Y_val, pred_bbc))
'''

#### 用Python API

In [ ]:
'''
import xgboost as xgb

dtrain = xgb.DMatrix(x_train_resam, label = y_train_resam)
num_xgb = 500

params = {
    'booster': 'gbtree',
    'eta': 0.2, 
    'eval_metric': 'logloss',
    'tree method': 'auto',
    'objective': 'binary:logistic'
}

bst = xgb.cv(params, dtrain, num_xgb, nfold = 5, verbose_eval = True)

dtest = xgb.DMatrix(test)
pred = bst.predict(dtest)
print(logloss()
'''

### LightGBM

In [4]:
import pickle

with open('lgb_values.pkl', 'rb') as f:
    x_train_resam = pickle.load(f)
    y_train_resam = pickle.load(f)
    test = pickle.load(f)

In [5]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_val, y_train, y_val = train_test_split(x_train_resam, y_train_resam, test_size = 0.2)
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary', 
    'metric': 'binary_logloss',
    'num_leaves': 3,
    'max_depth': 10,
    'min_data_in_leaf': 40,
    'learning_rate': 0.5,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'lambda_l2': 0.001,
    'verbose': 1
}


lgb_train = lgb.Dataset(X_train, label = y_train)
lgb_val = lgb.Dataset(X_val, y_val, reference = lgb_train)
gbm = lgb.train(params, lgb_train, num_boost_round = 2000, verbose_eval = True, valid_sets = lgb_val, early_stopping_rounds = None)
pred = gbm.predict(test, num_iteration = gbm.best_iteration)

importance = gbm.feature_importance()
names = gbm.feature_name()
for idx, im in sorted(enumerate(importance), key = lambda x: x[1], reverse = True):
    print('The score of {} is {}'.format(names[idx], im))

[1]	valid_0's binary_logloss: 0.662263
[2]	valid_0's binary_logloss: 0.647471
[3]	valid_0's binary_logloss: 0.639963
[4]	valid_0's binary_logloss: 0.634251
[5]	valid_0's binary_logloss: 0.629918
[6]	valid_0's binary_logloss: 0.626371
[7]	valid_0's binary_logloss: 0.623516
[8]	valid_0's binary_logloss: 0.619675
[9]	valid_0's binary_logloss: 0.617396
[10]	valid_0's binary_logloss: 0.615763
[11]	valid_0's binary_logloss: 0.614809
[12]	valid_0's binary_logloss: 0.613472
[13]	valid_0's binary_logloss: 0.612145
[14]	valid_0's binary_logloss: 0.611046
[15]	valid_0's binary_logloss: 0.609478
[16]	valid_0's binary_logloss: 0.608772
[17]	valid_0's binary_logloss: 0.607334
[18]	valid_0's binary_logloss: 0.605618
[19]	valid_0's binary_logloss: 0.604902
[20]	valid_0's binary_logloss: 0.604307
[21]	valid_0's binary_logloss: 0.603549
[22]	valid_0's binary_logloss: 0.603163
[23]	valid_0's binary_logloss: 0.602638
[24]	valid_0's binary_logloss: 0.600993
[25]	valid_0's binary_logloss: 0.600434
[26]	vali

[215]	valid_0's binary_logloss: 0.565252
[216]	valid_0's binary_logloss: 0.565165
[217]	valid_0's binary_logloss: 0.565183
[218]	valid_0's binary_logloss: 0.565154
[219]	valid_0's binary_logloss: 0.56515
[220]	valid_0's binary_logloss: 0.565128
[221]	valid_0's binary_logloss: 0.56511
[222]	valid_0's binary_logloss: 0.565073
[223]	valid_0's binary_logloss: 0.564936
[224]	valid_0's binary_logloss: 0.564907
[225]	valid_0's binary_logloss: 0.5649
[226]	valid_0's binary_logloss: 0.564851
[227]	valid_0's binary_logloss: 0.564742
[228]	valid_0's binary_logloss: 0.564704
[229]	valid_0's binary_logloss: 0.564661
[230]	valid_0's binary_logloss: 0.564566
[231]	valid_0's binary_logloss: 0.564364
[232]	valid_0's binary_logloss: 0.564185
[233]	valid_0's binary_logloss: 0.564156
[234]	valid_0's binary_logloss: 0.564065
[235]	valid_0's binary_logloss: 0.564026
[236]	valid_0's binary_logloss: 0.564011
[237]	valid_0's binary_logloss: 0.563955
[238]	valid_0's binary_logloss: 0.563978
[239]	valid_0's bina

[434]	valid_0's binary_logloss: 0.55871
[435]	valid_0's binary_logloss: 0.558697
[436]	valid_0's binary_logloss: 0.558657
[437]	valid_0's binary_logloss: 0.558618
[438]	valid_0's binary_logloss: 0.558594
[439]	valid_0's binary_logloss: 0.558567
[440]	valid_0's binary_logloss: 0.558561
[441]	valid_0's binary_logloss: 0.558457
[442]	valid_0's binary_logloss: 0.558436
[443]	valid_0's binary_logloss: 0.558462
[444]	valid_0's binary_logloss: 0.558473
[445]	valid_0's binary_logloss: 0.558457
[446]	valid_0's binary_logloss: 0.558489
[447]	valid_0's binary_logloss: 0.558535
[448]	valid_0's binary_logloss: 0.558474
[449]	valid_0's binary_logloss: 0.558462
[450]	valid_0's binary_logloss: 0.558501
[451]	valid_0's binary_logloss: 0.558517
[452]	valid_0's binary_logloss: 0.558547
[453]	valid_0's binary_logloss: 0.55852
[454]	valid_0's binary_logloss: 0.558535
[455]	valid_0's binary_logloss: 0.558555
[456]	valid_0's binary_logloss: 0.55857
[457]	valid_0's binary_logloss: 0.558542
[458]	valid_0's bin

[648]	valid_0's binary_logloss: 0.55639
[649]	valid_0's binary_logloss: 0.556365
[650]	valid_0's binary_logloss: 0.556381
[651]	valid_0's binary_logloss: 0.556377
[652]	valid_0's binary_logloss: 0.556368
[653]	valid_0's binary_logloss: 0.556323
[654]	valid_0's binary_logloss: 0.556316
[655]	valid_0's binary_logloss: 0.556315
[656]	valid_0's binary_logloss: 0.556334
[657]	valid_0's binary_logloss: 0.556336
[658]	valid_0's binary_logloss: 0.556334
[659]	valid_0's binary_logloss: 0.556328
[660]	valid_0's binary_logloss: 0.556343
[661]	valid_0's binary_logloss: 0.55633
[662]	valid_0's binary_logloss: 0.55632
[663]	valid_0's binary_logloss: 0.556311
[664]	valid_0's binary_logloss: 0.556321
[665]	valid_0's binary_logloss: 0.55634
[666]	valid_0's binary_logloss: 0.556294
[667]	valid_0's binary_logloss: 0.556319
[668]	valid_0's binary_logloss: 0.55634
[669]	valid_0's binary_logloss: 0.556291
[670]	valid_0's binary_logloss: 0.556282
[671]	valid_0's binary_logloss: 0.556258
[672]	valid_0's binar

[849]	valid_0's binary_logloss: 0.554671
[850]	valid_0's binary_logloss: 0.554655
[851]	valid_0's binary_logloss: 0.554648
[852]	valid_0's binary_logloss: 0.55467
[853]	valid_0's binary_logloss: 0.55465
[854]	valid_0's binary_logloss: 0.554629
[855]	valid_0's binary_logloss: 0.554618
[856]	valid_0's binary_logloss: 0.554593
[857]	valid_0's binary_logloss: 0.554599
[858]	valid_0's binary_logloss: 0.554588
[859]	valid_0's binary_logloss: 0.554558
[860]	valid_0's binary_logloss: 0.554582
[861]	valid_0's binary_logloss: 0.554568
[862]	valid_0's binary_logloss: 0.554564
[863]	valid_0's binary_logloss: 0.554563
[864]	valid_0's binary_logloss: 0.554532
[865]	valid_0's binary_logloss: 0.554524
[866]	valid_0's binary_logloss: 0.554491
[867]	valid_0's binary_logloss: 0.554504
[868]	valid_0's binary_logloss: 0.5545
[869]	valid_0's binary_logloss: 0.55449
[870]	valid_0's binary_logloss: 0.554492
[871]	valid_0's binary_logloss: 0.554531
[872]	valid_0's binary_logloss: 0.554557
[873]	valid_0's binar

[1052]	valid_0's binary_logloss: 0.554482
[1053]	valid_0's binary_logloss: 0.554446
[1054]	valid_0's binary_logloss: 0.554434
[1055]	valid_0's binary_logloss: 0.554357
[1056]	valid_0's binary_logloss: 0.55435
[1057]	valid_0's binary_logloss: 0.554346
[1058]	valid_0's binary_logloss: 0.554341
[1059]	valid_0's binary_logloss: 0.554335
[1060]	valid_0's binary_logloss: 0.554331
[1061]	valid_0's binary_logloss: 0.554322
[1062]	valid_0's binary_logloss: 0.554339
[1063]	valid_0's binary_logloss: 0.554342
[1064]	valid_0's binary_logloss: 0.554316
[1065]	valid_0's binary_logloss: 0.554332
[1066]	valid_0's binary_logloss: 0.55427
[1067]	valid_0's binary_logloss: 0.554281
[1068]	valid_0's binary_logloss: 0.554265
[1069]	valid_0's binary_logloss: 0.554265
[1070]	valid_0's binary_logloss: 0.554218
[1071]	valid_0's binary_logloss: 0.554241
[1072]	valid_0's binary_logloss: 0.554256
[1073]	valid_0's binary_logloss: 0.554266
[1074]	valid_0's binary_logloss: 0.554248
[1075]	valid_0's binary_logloss: 0.5

[1260]	valid_0's binary_logloss: 0.554314
[1261]	valid_0's binary_logloss: 0.55431
[1262]	valid_0's binary_logloss: 0.554298
[1263]	valid_0's binary_logloss: 0.55429
[1264]	valid_0's binary_logloss: 0.554279
[1265]	valid_0's binary_logloss: 0.554311
[1266]	valid_0's binary_logloss: 0.554317
[1267]	valid_0's binary_logloss: 0.554321
[1268]	valid_0's binary_logloss: 0.554286
[1269]	valid_0's binary_logloss: 0.554296
[1270]	valid_0's binary_logloss: 0.554293
[1271]	valid_0's binary_logloss: 0.554288
[1272]	valid_0's binary_logloss: 0.554296
[1273]	valid_0's binary_logloss: 0.554301
[1274]	valid_0's binary_logloss: 0.554319
[1275]	valid_0's binary_logloss: 0.554303
[1276]	valid_0's binary_logloss: 0.554272
[1277]	valid_0's binary_logloss: 0.554256
[1278]	valid_0's binary_logloss: 0.554209
[1279]	valid_0's binary_logloss: 0.554217
[1280]	valid_0's binary_logloss: 0.554219
[1281]	valid_0's binary_logloss: 0.554237
[1282]	valid_0's binary_logloss: 0.554232
[1283]	valid_0's binary_logloss: 0.5

[1467]	valid_0's binary_logloss: 0.554045
[1468]	valid_0's binary_logloss: 0.554043
[1469]	valid_0's binary_logloss: 0.554033
[1470]	valid_0's binary_logloss: 0.554031
[1471]	valid_0's binary_logloss: 0.554031
[1472]	valid_0's binary_logloss: 0.554024
[1473]	valid_0's binary_logloss: 0.553989
[1474]	valid_0's binary_logloss: 0.553982
[1475]	valid_0's binary_logloss: 0.55397
[1476]	valid_0's binary_logloss: 0.553983
[1477]	valid_0's binary_logloss: 0.553962
[1478]	valid_0's binary_logloss: 0.553956
[1479]	valid_0's binary_logloss: 0.553952
[1480]	valid_0's binary_logloss: 0.553964
[1481]	valid_0's binary_logloss: 0.553955
[1482]	valid_0's binary_logloss: 0.553941
[1483]	valid_0's binary_logloss: 0.553975
[1484]	valid_0's binary_logloss: 0.553994
[1485]	valid_0's binary_logloss: 0.554005
[1486]	valid_0's binary_logloss: 0.554011
[1487]	valid_0's binary_logloss: 0.554025
[1488]	valid_0's binary_logloss: 0.554043
[1489]	valid_0's binary_logloss: 0.554063
[1490]	valid_0's binary_logloss: 0.

[1670]	valid_0's binary_logloss: 0.55407
[1671]	valid_0's binary_logloss: 0.554072
[1672]	valid_0's binary_logloss: 0.554081
[1673]	valid_0's binary_logloss: 0.554107
[1674]	valid_0's binary_logloss: 0.554119
[1675]	valid_0's binary_logloss: 0.554083
[1676]	valid_0's binary_logloss: 0.554088
[1677]	valid_0's binary_logloss: 0.55406
[1678]	valid_0's binary_logloss: 0.554053
[1679]	valid_0's binary_logloss: 0.554093
[1680]	valid_0's binary_logloss: 0.554104
[1681]	valid_0's binary_logloss: 0.554047
[1682]	valid_0's binary_logloss: 0.554048
[1683]	valid_0's binary_logloss: 0.554044
[1684]	valid_0's binary_logloss: 0.554045
[1685]	valid_0's binary_logloss: 0.554024
[1686]	valid_0's binary_logloss: 0.554047
[1687]	valid_0's binary_logloss: 0.554059
[1688]	valid_0's binary_logloss: 0.554056
[1689]	valid_0's binary_logloss: 0.554037
[1690]	valid_0's binary_logloss: 0.554028
[1691]	valid_0's binary_logloss: 0.554023
[1692]	valid_0's binary_logloss: 0.554008
[1693]	valid_0's binary_logloss: 0.5

[1875]	valid_0's binary_logloss: 0.554173
[1876]	valid_0's binary_logloss: 0.554187
[1877]	valid_0's binary_logloss: 0.554183
[1878]	valid_0's binary_logloss: 0.554179
[1879]	valid_0's binary_logloss: 0.554186
[1880]	valid_0's binary_logloss: 0.554185
[1881]	valid_0's binary_logloss: 0.554161
[1882]	valid_0's binary_logloss: 0.554225
[1883]	valid_0's binary_logloss: 0.554229
[1884]	valid_0's binary_logloss: 0.554217
[1885]	valid_0's binary_logloss: 0.554215
[1886]	valid_0's binary_logloss: 0.554231
[1887]	valid_0's binary_logloss: 0.554231
[1888]	valid_0's binary_logloss: 0.554247
[1889]	valid_0's binary_logloss: 0.554276
[1890]	valid_0's binary_logloss: 0.554238
[1891]	valid_0's binary_logloss: 0.554231
[1892]	valid_0's binary_logloss: 0.55423
[1893]	valid_0's binary_logloss: 0.554237
[1894]	valid_0's binary_logloss: 0.554264
[1895]	valid_0's binary_logloss: 0.554251
[1896]	valid_0's binary_logloss: 0.554271
[1897]	valid_0's binary_logloss: 0.554273
[1898]	valid_0's binary_logloss: 0.

### DeepFM

#### 定义参数

In [ ]:
import tesorflow as tf
import numpy as np
import pandas as pd

dfm_params = {
    'use_fm': True,
    'use_deep': True,
    'embedding_size': 8,
    'dropout_fm': [1.0, 1.0],
    'deep_layers': [32, 32],
    'dropout_deep': [0.5, 0.5, 0.5],
    'deep_layer_activation': tf.nn.relu,
    'opoch': 30,
    'bach_size': 1024,
    'learning_rate': 0.001,
    'optimizer': 'adam',
    'batch_norm_dacay': 0.995,
    'l2_reg': 0.01,
    'verbose': True,
    'eval_metric': 'logloss',
    'random_seed': 100
}
dfm_params['feature_size'] = 
dfm_params['filed_szie'] = 

feat_index = tf.placeholder(tf.int32, shape = [None, None], name = 'feat_index')
feat_value = tf.placeholder(tf.float32, shape = [None, None], name = 'feat_value')

label = tf.placeholder(tf.float32, shape = [None, 1], name = 'label')

weights = dict()

##embeddings
weights['feature_embeddings'] = tf.Variable(
    tf.random_normal([dfm_params['feature_size'], dfm_params['embedding_size']], 0.0, 0.01), name = 'feature_embeddings')
weights['feature_bias'] = tf.Variable(
    tf.ramdom_normal([dfm_params['feature_size'], 1], 0.0, 0.01), name = 'feature_bias')###weights[''feature_bias]是一次项的权重

##deep layers
num_layer = len(dfm_params['deep_layers'])
input_size = dfm_params['field_size'] * dfm_params['embedding_size']
glorot = np.sqrt(2.0 / (input_size + dfm_params['deep_layers'][0]))

weights['layer_0'] = tf.Variable(
    np.random.normal(loc = 0, scale = glorot, size = (input_size, dfm_params['deep_layers'][0])), dtype = np.float32)
weights['bias_0'] = tf.Variable(
    np.random.normal(loc = 0, scale = glorot, size = (1, dfm_params['deep_layers'][0])), dtype = np.float32)

for i in range(1, num_layer):
    glorot = np.sqrt(2.0 / (dfm_params['deep_layers'][i - 1] + dfm_params['deep_layer'][i]))
    weights['layer_%d'%(i)] = tf.Variable(
        np.random.normal(loc = 0, scale = glorot, size = (dfm_params['deep_layers'][i - 1], dfm_params['deep_layers'][i])), dtype = np.float32)
    weights['bias_%d'%(i)] = tf.Variable(
        np.ransom.normal(loc = 0, scale = glorot, size = (1, dfm_params['deep_layers'][i])), dtype = np.float32)

##concat layer
if dfm_params['use_fm'] and dfm_params['use_deep']:
    input_size = dfm_params['field_size'] + dfm_params['embedding_size'] + dfm_params['deep_layers'][-1]
elif dfm_params['use_fm']:
    input_size = dfm_params['field_size'] + dfm_params['embedding_size']
else:    
    input_size = dfm_params['deep_layers'][-1]

glorot = np.sqrt(2.0 / (input_size + 1))
weights['concat_projection'] = tf.Variable(np.random.normal(loc = 0, scale = glorot, size = [input_size, 1]), dtype = np.float32)
weights['concat_bias'] = tf.Variable(tf.constant(0.01), dtype = np.float32)

#### 构造模型

In [ ]:
##embeddings
embeddings = tf.nn.embedding_lookup(weights['feature_embeddings'], feat_index)
reshaped_feat_value = tf.reshape(feat_value, shape = [-1, dfm_params['field_size'], 1])
print(reshaped_feat_value)
embeddings = tf.multiply(embeddings, reshaped_feat_value)

##fm
fm_first_order = tf.nn.embedding_lookup(weights['feature_bias'], feat_index)
fm_first_order = tf.reduce_sum(tf.multiply(fm_first_order, reshaped_feat_value), 2)

summed_features_emb = tf.reduce_sum(embeddings, 1)
summed_features_emb_square = tf.square(summed_features_emb)

squared_features_emb = tf.square(embeddings)
squared_feateres_emb_sum = tf.reduce_sum(squared_features_emb, 1)

fm_second_order = 0.5 * tf.subtract(summed_features_emb_square, squared_features_emb_sum)

##deep
y_deep = tf.reshape(embeddings, shape = [-1, dfm_params['field_size'] * dfm_params['embeddings_size']])

for i in range(len(dfm_params['deep_layers'])):
    y_deep = tf.add(tf.matmul(y_deep, weights['layer_%d'%(i)]), weights['bias%d'%(i)])
    y_deep = tf.nn.relu(y_deep)
    
##final layer
if dfm_params['use_fm'] and dfm_params['use_deep']:
    concat_input = tf.concat([fm_first_order, fm_second_order, y_deep], axis = 1)
elif dfm_params['use_fm']:
    concat_input = tf.concat([fm_first_order, fm_second_order], axis = 1)
else:
    concat_input = y_deep

out = tf.nn.sigmoid(tf.add(tf.matmul(concat_input, weights['concat_projection']), weights['concat_bias']))

loss = tf.losses.log_loss(tf.reshape(label, (-1,1)), out)
optimizer = tf.train.AdamOptimizer(learning_rate = dfm_params['learning_rate'], beta1 = 0.9, beta2 = 0.999, epslion = 1e-8).minimize(loss)

#### 训练模型

In [ ]:
with tf.Sesstion() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        epoch_loss, =sess.run([loss, optimizer], feed_dict{feat_index: , 
                                                          feat_value: ,label: y_train})
        print('epoch%s ,loss is %s'%(str(i), str(epoch_loss)))